In [ ]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
import pytz

In [ ]:
import os
import numpy as np
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/master project/data/GOOG' #เข้า drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api_key = 'J4AM5YKF88UA41WO'

In [ ]:
def _get_data(symbols, time_from, time_to, api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

In [ ]:
def get_dataset(time_from="20190101T0000",
                time_to="20250301T0000",
                MAX_API_CALLS_PER_DAY = 99999, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 300 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%600==0:
            time.sleep(65)

        data=_get_data('GOOG',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract GOOG specific relevance (we didn't use it in video)

    # Only take tickers with GOOG in headline
    df=df[df.title.str.contains('GOOG|google|Alphabet',case=False)]
    # Only take when # of tickers = 1
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    #df.set_index('time_published',inplace=True)
    # Sort by time published
    #df.sort_index(inplace=True)
    return df


In [ ]:
df = get_dataset()

In [ ]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.tz_localize('UTC')

In [ ]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
2,Alphabet Stock Enters Oversold Territory - Is ...,https://www.benzinga.com/news/large-cap/25/02/...,2025-02-28 21:47:23,[Erica Kollmann],Alphabet faces multiple headwinds that may hav...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.100324,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.2717..."
4,3 Reasons Why the Chip Stock Rally Is Over - F...,https://www.benzinga.com/25/02/44055593/3-reas...,2025-02-28 19:39:09,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.074936,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.07339..."
5,Time to Sell Chip Stocks as the Rally is Over ...,https://www.benzinga.com/trading-ideas/25/02/4...,2025-02-28 19:39:01,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.072516,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0490..."
6,DeliverHealth and Google Cloud Partner to Brin...,https://www.benzinga.com/pressreleases/25/02/n...,2025-02-28 19:00:00,[PRNewswire],"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",https://mma.prnewswire.com/media/2341454/Deliv...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.500734,Bullish,"[{'ticker': 'DTST', 'relevance_score': '0.0780..."
25,TikTok Invests $8.8 Billion in Thailand Data C...,https://www.benzinga.com/markets/emerging-mark...,2025-02-28 13:09:38,[Anusuya Lahiri],TikTok commits $8.8 billion to Thailand data c...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.200963,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.3456..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...",https://www.globenewswire.com/news-release/202...,2022-03-07 14:32:00,[Swifty Global],"NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.407720,Bullish,"[{'ticker': 'DRCR', 'relevance_score': '0.1078..."
231,Google will use office space in Poland to supp...,https://www.cnbc.com/2022/03/07/google-will-us...,2022-03-07 14:11:55,[Sarah Alessandrini],Google will open its offices in Poland as a re...,https://image.cnbcfm.com/api/v1/image/10702616...,CNBC,Top News,www.cnbc.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.106686,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.7212..."
239,"Apple, Google, Microsoft, and other tech giant...",https://www.fastcompany.com/90728462/apple-goo...,2022-03-07 12:55:47,[],Return-to-office update: list of tech companie...,https://images.fastcompany.net/image/upload/w_...,Fast Company,n/a,www.fastcompany.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.245551,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.159463..."
280,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...",https://www.zacks.com/stock/news/1877591/will-...,2022-03-04 15:19:00,[Zacks Investment Research],"Let's sneak a peek into how Alphabet (GOOGL), ...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.273713,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.158938..."


In [ ]:
df['time_published']=df['time_published'].dt.tz_convert('US/Eastern')

In [ ]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
2,Alphabet Stock Enters Oversold Territory - Is ...,https://www.benzinga.com/news/large-cap/25/02/...,2025-02-28 16:47:23-05:00,[Erica Kollmann],Alphabet faces multiple headwinds that may hav...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.100324,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.2717..."
4,3 Reasons Why the Chip Stock Rally Is Over - F...,https://www.benzinga.com/25/02/44055593/3-reas...,2025-02-28 14:39:09-05:00,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.074936,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.07339..."
5,Time to Sell Chip Stocks as the Rally is Over ...,https://www.benzinga.com/trading-ideas/25/02/4...,2025-02-28 14:39:01-05:00,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.072516,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0490..."
6,DeliverHealth and Google Cloud Partner to Brin...,https://www.benzinga.com/pressreleases/25/02/n...,2025-02-28 14:00:00-05:00,[PRNewswire],"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",https://mma.prnewswire.com/media/2341454/Deliv...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.500734,Bullish,"[{'ticker': 'DTST', 'relevance_score': '0.0780..."
25,TikTok Invests $8.8 Billion in Thailand Data C...,https://www.benzinga.com/markets/emerging-mark...,2025-02-28 08:09:38-05:00,[Anusuya Lahiri],TikTok commits $8.8 billion to Thailand data c...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.200963,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.3456..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...",https://www.globenewswire.com/news-release/202...,2022-03-07 09:32:00-05:00,[Swifty Global],"NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.407720,Bullish,"[{'ticker': 'DRCR', 'relevance_score': '0.1078..."
231,Google will use office space in Poland to supp...,https://www.cnbc.com/2022/03/07/google-will-us...,2022-03-07 09:11:55-05:00,[Sarah Alessandrini],Google will open its offices in Poland as a re...,https://image.cnbcfm.com/api/v1/image/10702616...,CNBC,Top News,www.cnbc.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.106686,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.7212..."
239,"Apple, Google, Microsoft, and other tech giant...",https://www.fastcompany.com/90728462/apple-goo...,2022-03-07 07:55:47-05:00,[],Return-to-office update: list of tech companie...,https://images.fastcompany.net/image/upload/w_...,Fast Company,n/a,www.fastcompany.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.245551,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.159463..."
280,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...",https://www.zacks.com/stock/news/1877591/will-...,2022-03-04 10:19:00-05:00,[Zacks Investment Research],"Let's sneak a peek into how Alphabet (GOOGL), ...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.273713,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.158938..."


In [ ]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.strftime('%Y-%m-%d')

In [ ]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
2,Alphabet Stock Enters Oversold Territory - Is ...,https://www.benzinga.com/news/large-cap/25/02/...,2025-02-28,[Erica Kollmann],Alphabet faces multiple headwinds that may hav...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.100324,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.2717..."
4,3 Reasons Why the Chip Stock Rally Is Over - F...,https://www.benzinga.com/25/02/44055593/3-reas...,2025-02-28,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.074936,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.07339..."
5,Time to Sell Chip Stocks as the Rally is Over ...,https://www.benzinga.com/trading-ideas/25/02/4...,2025-02-28,[Nic Chahine],The semiconductor market has always been ripe ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.072516,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0490..."
6,DeliverHealth and Google Cloud Partner to Brin...,https://www.benzinga.com/pressreleases/25/02/n...,2025-02-28,[PRNewswire],"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",https://mma.prnewswire.com/media/2341454/Deliv...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.500734,Bullish,"[{'ticker': 'DTST', 'relevance_score': '0.0780..."
25,TikTok Invests $8.8 Billion in Thailand Data C...,https://www.benzinga.com/markets/emerging-mark...,2025-02-28,[Anusuya Lahiri],TikTok commits $8.8 billion to Thailand data c...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.200963,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.3456..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...",https://www.globenewswire.com/news-release/202...,2022-03-07,[Swifty Global],"NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.407720,Bullish,"[{'ticker': 'DRCR', 'relevance_score': '0.1078..."
231,Google will use office space in Poland to supp...,https://www.cnbc.com/2022/03/07/google-will-us...,2022-03-07,[Sarah Alessandrini],Google will open its offices in Poland as a re...,https://image.cnbcfm.com/api/v1/image/10702616...,CNBC,Top News,www.cnbc.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.106686,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.7212..."
239,"Apple, Google, Microsoft, and other tech giant...",https://www.fastcompany.com/90728462/apple-goo...,2022-03-07,[],Return-to-office update: list of tech companie...,https://images.fastcompany.net/image/upload/w_...,Fast Company,n/a,www.fastcompany.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.245551,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.159463..."
280,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...",https://www.zacks.com/stock/news/1877591/will-...,2022-03-04,[Zacks Investment Research],"Let's sneak a peek into how Alphabet (GOOGL), ...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.273713,Somewhat-Bearish,"[{'ticker': 'FB', 'relevance_score': '0.158938..."


In [ ]:
News_GOOG = df[["time_published", "title", "summary","overall_sentiment_score"]]

In [ ]:
News_GOOG

,time_published,title,summary,overall_sentiment_score
2,2025-02-28,Alphabet Stock Enters Oversold Territory - Is ...,Alphabet faces multiple headwinds that may hav...,0.100324
4,2025-02-28,3 Reasons Why the Chip Stock Rally Is Over - F...,The semiconductor market has always been ripe ...,0.074936
5,2025-02-28,Time to Sell Chip Stocks as the Rally is Over ...,The semiconductor market has always been ripe ...,0.072516
6,2025-02-28,DeliverHealth and Google Cloud Partner to Brin...,"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",0.500734
25,2025-02-28,TikTok Invests $8.8 Billion in Thailand Data C...,TikTok commits $8.8 billion to Thailand data c...,0.200963
...,...,...,...,...
226,2022-03-07,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...","NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",0.407720
231,2022-03-07,Google will use office space in Poland to supp...,Google will open its offices in Poland as a re...,0.106686
239,2022-03-07,"Apple, Google, Microsoft, and other tech giant...",Return-to-office update: list of tech companie...,-0.245551
280,2022-03-04,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...","Let's sneak a peek into how Alphabet (GOOGL), ...",-0.273713


In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
sia = SentimentIntensityAnalyzer()
# Analyze sentiment for each summary

x = str(News_GOOG['summary']).encode("utf-8")
News_GOOG['sentiment_score'] = News_GOOG['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])

<ipython-input-32-22b72e2ece91>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  News_GOOG['sentiment_score'] = News_GOOG['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [ ]:
News_GOOG

,time_published,title,summary,overall_sentiment_score,sentiment_score
2,2025-02-28,Alphabet Stock Enters Oversold Territory - Is ...,Alphabet faces multiple headwinds that may hav...,0.100324,0.1531
4,2025-02-28,3 Reasons Why the Chip Stock Rally Is Over - F...,The semiconductor market has always been ripe ...,0.074936,0.8042
5,2025-02-28,Time to Sell Chip Stocks as the Rally is Over ...,The semiconductor market has always been ripe ...,0.072516,0.2592
6,2025-02-28,DeliverHealth and Google Cloud Partner to Brin...,"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",0.500734,0.7430
25,2025-02-28,TikTok Invests $8.8 Billion in Thailand Data C...,TikTok commits $8.8 billion to Thailand data c...,0.200963,0.3612
...,...,...,...,...,...
226,2022-03-07,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...","NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",0.407720,0.7964
231,2022-03-07,Google will use office space in Poland to supp...,Google will open its offices in Poland as a re...,0.106686,0.0000
239,2022-03-07,"Apple, Google, Microsoft, and other tech giant...",Return-to-office update: list of tech companie...,-0.245551,0.0000
280,2022-03-04,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...","Let's sneak a peek into how Alphabet (GOOGL), ...",-0.273713,-0.1280


In [ ]:
News_GOOG_Nozero=News_GOOG.loc[News_GOOG['sentiment_score'] != 0]
News_GOOG_Nozero

,time_published,title,summary,overall_sentiment_score,sentiment_score
2,2025-02-28,Alphabet Stock Enters Oversold Territory - Is ...,Alphabet faces multiple headwinds that may hav...,0.100324,0.1531
4,2025-02-28,3 Reasons Why the Chip Stock Rally Is Over - F...,The semiconductor market has always been ripe ...,0.074936,0.8042
5,2025-02-28,Time to Sell Chip Stocks as the Rally is Over ...,The semiconductor market has always been ripe ...,0.072516,0.2592
6,2025-02-28,DeliverHealth and Google Cloud Partner to Brin...,"MADISON, Wis., Feb. 28, 2025 /PRNewswire/ -- D...",0.500734,0.7430
25,2025-02-28,TikTok Invests $8.8 Billion in Thailand Data C...,TikTok commits $8.8 billion to Thailand data c...,0.200963,0.3612
...,...,...,...,...,...
221,2022-03-07,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZ...",Alphabet's (GOOGL) Google peps up audio stream...,0.114362,0.1779
225,2022-03-07,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...","NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",0.404223,0.7964
226,2022-03-07,"OTC: DRCR, Swifty Global ( Dear Cashmere Hold...","NEW YORK, NY, March 07, 2022 ( GLOBE NEWSWI...",0.407720,0.7964
280,2022-03-04,"Will Russia-Ukraine War Hurt Growth of GOOGL, ...","Let's sneak a peek into how Alphabet (GOOGL), ...",-0.273713,-0.1280


In [ ]:
daily_avg_sentiment_GOOG = News_GOOG.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_GOOG

,time_published,sentiment_score
0,2022-03-02,0.177900
1,2022-03-04,-0.128000
2,2022-03-07,0.252255
3,2022-03-08,0.337384
4,2022-03-09,0.226725
...,...,...
1053,2025-02-24,0.261714
1054,2025-02-25,0.250000
1055,2025-02-26,0.371140
1056,2025-02-27,0.458250


In [ ]:
daily_avg_sentiment_Nozero_GOOG = News_GOOG_Nozero.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_Nozero_GOOG

,time_published,sentiment_score
0,2022-03-02,0.177900
1,2022-03-04,-0.128000
2,2022-03-07,0.396400
3,2022-03-08,0.468589
4,2022-03-09,0.302300
...,...,...
1037,2025-02-24,0.366400
1038,2025-02-25,0.250000
1039,2025-02-26,0.371140
1040,2025-02-27,0.458250


In [ ]:
daily_avg_sentiment_GOOG.to_csv(os.path.join(path,'daily_avg_sentiment.csv'))
News_GOOG.to_csv(os.path.join(path,'News_GOOG.csv'))

In [ ]:
daily_avg_sentiment_Nozero_GOOG.to_csv(os.path.join(path,'daily_avg_sentiment_Nozero.csv'))
News_GOOG_Nozero.to_csv(os.path.join(path,'News_GOOG_Nozero.csv'))